<a href="https://colab.research.google.com/github/NandoV179/SystemEstimateSpeedOfVehicles/blob/main/match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
secuencia="40871"
data = pd.read_csv('XML_'+secuencia+'.txt', header=0, sep=',')
data1 = pd.read_csv('RCNN_RED_'+secuencia+'.txt', header=0, sep=',')


print(data)
print(data1)


       frame  Id       X        Y       W        H  Velocidad  densidad
0          1   1  756.27  167.470  130.06   70.330    1.71000        16
1          1   2  570.67  136.530   97.00   54.340    1.75920        16
2          1   3  430.93  119.470   61.80   35.130    1.38670        16
3          1   4  330.67   96.000   49.00   30.870    0.75425        16
4          1   5  467.20  354.130  223.93  186.870    6.73610        16
...      ...  ..     ...      ...     ...      ...        ...       ...
36620   1720  33  309.33  147.200   74.60   50.070    0.00000        22
36621   1720  34  460.80  103.470   76.73   55.400    0.00000        22
36622   1720  35  269.87   93.867   42.60   31.933    0.00000        22
36623   1720  36  231.47   83.200   38.33   28.730    0.00000        22
36624   1720  37  217.60  108.800   47.93   36.200    0.00000        22

[36625 rows x 8 columns]
       frame        x       y        w       h  densidad
0          2  207.040  156.56   46.880   40.92       

In [ ]:
R=np.array(data.to_numpy())
Y=H=np.array(data1.to_numpy())
filasR=np.shape(R)
filaR=filasR[0]
print(filaR)
filasY=np.shape(Y)
filaY=filasY[0]
print(filaY)

36625
53423


In [ ]:
cont_no_match=0
l_iou=[]
l_frame=[]
l_idCar=[]
l_x=[]
l_y=[]
l_w=[]
l_h=[]
l_vel=[]

ln_frame=[]
ln_x=[]
ln_y=[]
ln_w=[]
ln_h=[]
for i in range (0 , filaY):
  Yfr = Y[i][0]
  Yx=Y[i][1]
  Yy=Y[i][2]
  Yw=Y[i][3]
  Yh=Y[i][4]
  Yd=Y[i][5]
  cont=0
  for j in range (0,filaR):
    Rfr= R[j][0]
    Rid = R[j][1]
    Rx =R[j][2]
    Ry= R[j][3]
    Rw= R[j][4]
    Rh= R[j][5]
    Rvel= R[j][6]
    den=R[j][7]
    densidad=int(den)

    if (Yfr == Rfr):

      box1 =[[Rx,Ry+Rh],[Rx+Rw,Ry+Rh],[Rx+Rw,Ry],[Rx,Ry]]
      box2 =[[Yx,Yy+Yh],[Yx+Yw,Yy+Yh],[Yx+Yw,Yy],[Yx,Yy]]

      #print(box1 , box2)
      poly_1 = Polygon(box1)
      poly_2 = Polygon(box2)
      
      iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
      
      if (iou > 0.7):
        #print(iou, Yfr, Yx, Yy, Yw, Yh)
        #print(iou, Rfr, Rx, Ry, Rw, Rh, Rid, Rvel )
        l_iou.append(iou)
        l_frame.append(Yfr)
        l_idCar.append(Rid)
        l_x.append(Yx)
        l_y.append(Yy)
        l_w.append(Yw)
        l_h.append(Yh)
        l_vel.append(Rvel)
        break
      else:
        cont = cont + 1
      
      if (cont == densidad):
        cont_no_match = cont_no_match + 1
        #print("no match",Yfr, Yx, Yy, Yw, Yh )
        ln_frame.append(Yfr)
        ln_x.append(Yx)
        ln_y.append(Yy)
        ln_w.append(Yw)
        ln_h.append(Yh)


df = pd.DataFrame({'IOU': l_iou,
                   'frame': l_frame,
                   'id': l_idCar,
                   'X': l_x,
                   'Y': l_y,
                   'W': l_w,
                   'H': l_h,
                   'Velocidad': l_vel,
                   })
nombreArchivo = 'YOLO_Com_MVI_' + secuencia + '.txt'
df.to_csv(nombreArchivo, index=False)

df1 = pd.DataFrame({'frame': ln_frame,
                   'X': ln_x,
                   'Y': ln_y,
                   'W': ln_w,
                   'H': ln_h,
                   })
nombreArchivo1 = 'YOLO_No_Match_MVI_' + secuencia + '.txt'
df1.to_csv(nombreArchivo1, index=False)

